# Optimisation hyperparamètres

Objectif : minimiser le coût métier (FN*10 + FP*1) via OOF CV


## Import

## Config PATH + MLFLOW

In [1]:
import os
import sys
from pathlib import Path


CWD = Path.cwd()
PROJECT_ROOT = CWD.parent.parent
DB_PATH = (PROJECT_ROOT / "mlflow.db").resolve()
ARTIFACT_ROOT = (PROJECT_ROOT / "artifacts").resolve()
ARTIFACT_ROOT.mkdir(parents=True, exist_ok=True)

FEATURE_REDUCTION_DIR = PROJECT_ROOT / "reports" / "feature_reduction"
FEATURE_REDUCTION_DIR.mkdir(parents=True, exist_ok=True)

os.environ["MLFLOW_TRACKING_URI"] = f"sqlite:///{DB_PATH.as_posix()}"
os.environ["MLFLOW_ARTIFACT_URI"] = ARTIFACT_ROOT.as_uri()


sys.path.append(str(PROJECT_ROOT))

import mlflow  


mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])

print("CWD =", CWD)
print("Tracking URI =", mlflow.get_tracking_uri())
print("Artifacts root (env) =", os.environ["MLFLOW_ARTIFACT_URI"])


CWD = c:\Users\yoann\Documents\open classrooms\projet 8\livrables\pret a dépenser\notebooks\03_modeling
Tracking URI = sqlite:///C:/Users/yoann/Documents/open classrooms/projet 8/livrables/pret a dépenser/mlflow.db
Artifacts root (env) = file:///C:/Users/yoann/Documents/open%20classrooms/projet%208/livrables/pret%20a%20d%C3%A9penser/artifacts


In [2]:
import pandas as pd
from catboost import CatBoostClassifier
import numpy as np
from src.modeling.train import train_with_cv
from src.modeling.prepare_for_model import prepare_application_for_model
from src.tracking import mlflow_tracking
import  json
from datetime import datetime
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    roc_auc_score,
    confusion_matrix,
    recall_score,
    precision_score,
    f1_score,
    fbeta_score,
)

EXPERIMENT_NAME = "home_credit_hyperparam_cb"
exp_id = mlflow_tracking.get_or_create_experiment(EXPERIMENT_NAME, ARTIFACT_ROOT)
mlflow.set_experiment(EXPERIMENT_NAME)
#mlflow ui --backend-store-uri sqlite:///mlflow.db

c:\Users\yoann\Documents\open classrooms\projet 8\livrables\pret a dépenser\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<Experiment: artifact_location='file:///C:/Users/yoann/Documents/open%20classrooms/projet%208/livrables/pret%20a%20d%C3%A9penser/artifacts', creation_time=1771241287787, experiment_id='4', last_update_time=1771241287787, lifecycle_stage='active', name='home_credit_hyperparam_cb', tags={}>

## Chargement du dataset + sélection feature set (kept_features_*.txt)

In [3]:

sys.path.append(str((PROJECT_ROOT / "src").as_posix()))
from modeling.prepare_for_model import prepare_application_for_model

TRAIN_PATH = PROJECT_ROOT / "data" / "processed" / "train_split.csv"
df_train = pd.read_csv(TRAIN_PATH)

# format "boosting"
X_full, y = prepare_application_for_model(df_train, model_type="boosting")
print("TRAIN full:", X_full.shape, "| y:", y.shape)

FEATURE_REDUCTION_DIR = PROJECT_ROOT / "reports" / "feature_reduction"
FEATURE_SET_NAME = "top125_nocorr"
kept_file = FEATURE_REDUCTION_DIR / f"kept_features_{FEATURE_SET_NAME}.txt"

if not kept_file.exists():
    raise FileNotFoundError(f"Fichier introuvable: {kept_file}")

kept_features = [
    l.strip()
    for l in kept_file.read_text(encoding="utf-8").splitlines()
    if l.strip()
]
kept_features = [c for c in kept_features if c in X_full.columns]
if len(kept_features) == 0:
    raise ValueError(f"kept_features vide -> vérifie le contenu de: {kept_file}")

X = X_full[kept_features].copy()
print("TRAIN reduced:", X.shape)
print("feature_set  :", FEATURE_SET_NAME)
print("kept_file    :", kept_file)

TRAIN full: (215257, 1656) | y: (215257,)
TRAIN reduced: (215257, 125)
feature_set  : top125_nocorr
kept_file    : c:\Users\yoann\Documents\open classrooms\projet 8\livrables\pret a dépenser\reports\feature_reduction\kept_features_top125_nocorr.txt


## Paramètres + lock

In [4]:

REPORTS_DIR = PROJECT_ROOT / "reports" / "hyperparameter_optimization"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

N_SPLITS = 3
RANDOM_STATE = 42

THRESH_FIXED = 0.5
COST_FN = 10
COST_FP = 1
FBETA_BETA = 3

lock = {
    "created_at": datetime.now().isoformat(timespec="seconds"),
    "dataset_path": str(TRAIN_PATH),
    "dataset_rows": int(X_full.shape[0]),
    "dataset_features_full": int(X_full.shape[1]),
    "feature_set": FEATURE_SET_NAME,
    "feature_set_source": str(kept_file),
    "feature_set_n": int(X.shape[1]),
    "cv": {"n_splits": int(N_SPLITS), "random_state": int(RANDOM_STATE)},
    "business": {
        "threshold_fixed": float(THRESH_FIXED),
        "cost_fn": int(COST_FN),
        "cost_fp": int(COST_FP),
        "fbeta_beta": float(FBETA_BETA),
    },
}

lock_path = REPORTS_DIR / f"hyperparam_dataset_lock_cb_{FEATURE_SET_NAME}.json"
lock_path.write_text(json.dumps(lock, ensure_ascii=False, indent=2), encoding="utf-8")
print("Lock saved:", lock_path)

Lock saved: c:\Users\yoann\Documents\open classrooms\projet 8\livrables\pret a dépenser\reports\hyperparameter_optimization\hyperparam_dataset_lock_cb_top125_nocorr.json


## Métriques + coût métier

In [5]:
from src.modeling.metrics import compute_metrics_and_cost


## Préparation CatBoost (cat -> str + gestion NA)

In [6]:
from src.modeling.prepare_catboost import prepare_catboost

In [7]:
from src.modeling.cv import cv_oof_proba_cb

## Modèle + petite grille Optuna

In [8]:

MODEL_NAME = "CatBoost"

base_params = {
    "loss_function": "Logloss",
    "random_seed": RANDOM_STATE,
    "auto_class_weights": "Balanced",
    "allow_writing_files": False,  # évite logs locaux
}

from src.modeling.hyperopt import suggest_params


## Objective Optuna

In [9]:
from src.modeling.hyperopt import make_objective
import optuna

objective = make_objective(
    X=X,
    y=y,
    base_params=base_params,
    MODEL_NAME=MODEL_NAME,
    FEATURE_SET_NAME=FEATURE_SET_NAME,
    N_SPLITS=N_SPLITS,
    RANDOM_STATE=RANDOM_STATE,
    THRESH_FIXED=THRESH_FIXED,
    COST_FN=COST_FN,
    COST_FP=COST_FP,
    FBETA_BETA=FBETA_BETA,
    cv_oof_proba_fn=cv_oof_proba_cb,
)


## Lancement optimisation

In [10]:

N_TRIALS = 100

study = optuna.create_study(
    direction="minimize",
    study_name=f"{MODEL_NAME}_cost_min_{FEATURE_SET_NAME}",
    sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE),
)
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

print("Best trial:", study.best_trial.number)
print("Best cost :", study.best_value)
print("Best params:")
for k, v in study.best_params.items():
    print(f"  - {k}: {v}")

[I 2026-02-16 12:28:49,044] A new study created in memory with name: CatBoost_cost_min_top125_nocorr
Best trial: 0. Best value: 106761:   1%|          | 1/100 [03:23<5:36:28, 203.92s/it]

[I 2026-02-16 12:32:12,981] Trial 0 finished with value: 106761.0 and parameters: {'iterations': 350, 'learning_rate': 0.09237421878009823, 'depth': 7, 'l2_leaf_reg': 3.968793330444372, 'subsample': 0.7468055921327309, 'rsm': 0.7467983561008608}. Best is trial 0 with value: 106761.0.


Best trial: 1. Best value: 106220:   2%|▏         | 2/100 [05:55<4:42:56, 173.23s/it]

[I 2026-02-16 12:34:44,727] Trial 1 finished with value: 106220.0 and parameters: {'iterations': 223, 'learning_rate': 0.0806234057607385, 'depth': 7, 'l2_leaf_reg': 5.105903209394756, 'subsample': 0.7061753482887407, 'rsm': 0.9909729556485982}. Best is trial 1 with value: 106220.0.


Best trial: 1. Best value: 106220:   3%|▎         | 3/100 [09:33<5:13:11, 193.73s/it]

...

[I 2026-02-16 18:58:08,086] Trial 97 finished with value: 105303.0 and parameters: {'iterations': 554, 'learning_rate': 0.07182044763615973, 'depth': 5, 'l2_leaf_reg': 9.26538525282765, 'subsample': 0.9668258465664425, 'rsm': 0.770900669070188}. Best is trial 67 with value: 105075.0.


Best trial: 67. Best value: 105075:  99%|█████████▉| 99/100 [6:33:44<04:25, 265.20s/it]

[I 2026-02-16 19:02:33,537] Trial 98 finished with value: 105717.0 and parameters: {'iterations': 575, 'learning_rate': 0.07326544184405956, 'depth': 5, 'l2_leaf_reg': 9.390439752453318, 'subsample': 0.9570398299943016, 'rsm': 0.7713416669495847}. Best is trial 67 with value: 105075.0.


Best trial: 67. Best value: 105075: 100%|██████████| 100/100 [6:38:11<00:00, 238.92s/it]

[I 2026-02-16 19:07:00,915] Trial 99 finished with value: 105457.0 and parameters: {'iterations': 562, 'learning_rate': 0.0792070326910723, 'depth': 5, 'l2_leaf_reg': 9.934955582455434, 'subsample': 0.9675930301355375, 'rsm': 0.7989914106595668}. Best is trial 67 with value: 105075.0.
Best trial: 67
Best cost : 105075.0
Best params:
  - iterations: 586
  - learning_rate: 0.055885310521729084
  - depth: 5
  - l2_leaf_reg: 4.441358316524765
  - subsample: 0.9132503854207161
  - rsm: 0.7320529782078392


In [11]:

best_payload = {
    "created_at": datetime.now().isoformat(timespec="seconds"),
    "experiment": EXPERIMENT_NAME,
    "model_name": MODEL_NAME,
    "feature_set": FEATURE_SET_NAME,
    "feature_set_source": str(kept_file),
    "objective": "minimize_business_cost",
    "threshold_fixed": THRESH_FIXED,
    "cost_fn": COST_FN,
    "cost_fp": COST_FP,
    "fbeta_beta": FBETA_BETA,
    "n_features": int(X.shape[1]),
    "n_splits": int(N_SPLITS),
    "random_state": int(RANDOM_STATE),
    "best_value": float(study.best_value),
    "best_params": study.best_params,
}

best_path = REPORTS_DIR / f"optuna_best_{MODEL_NAME.lower()}_{FEATURE_SET_NAME}.json"
best_path.write_text(json.dumps(best_payload, ensure_ascii=False, indent=2), encoding="utf-8")
print("Saved:", best_path)

with mlflow.start_run(run_name=f"{MODEL_NAME}_optuna_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
    mlflow.set_tag("phase", "hyperparam_optuna_summary")
    mlflow.set_tag("model_name", MODEL_NAME)
    mlflow.set_tag("dataset", "train_split")
    mlflow.set_tag("feature_set", FEATURE_SET_NAME)
    mlflow.set_tag("best_trial_number", str(study.best_trial.number))

    mlflow.log_metric("best_business_cost", float(study.best_value))
    mlflow.log_params({f"best.{k}": v for k, v in study.best_params.items()})

    mlflow.log_artifact(str(best_path))
    mlflow.log_artifact(str(lock_path))
    mlflow.log_artifact(str(kept_file))

print("Optuna tuning terminé.")

Saved: c:\Users\yoann\Documents\open classrooms\projet 8\livrables\pret a dépenser\reports\hyperparameter_optimization\optuna_best_catboost_top125_nocorr.json
Optuna tuning terminé.
